In [26]:
from allensdk.api.queries.reference_space_api import ReferenceSpaceApi
import nibabel as nib
import nrrd
import numpy as np
import os
from pathlib import Path
import skimage.transform as skit

ALLEN_RESOLUTIONS = [10, 25, 50, 100]


def download_allen_template(res=100, directory="."):
    """Downloading the Allen mouse brain template volume at a given resolution, and setting the correct affine
    Parameters
    ----------
    res : int
        Reference volume resolution. (Available resolutions : 10, 25, 50, 100 microns)
    directory : str
        Directory where the data will be saved.
    Returns
    -------
    nifti_file : str
        Absolute path to the downloaded template (nifti format, aligned to RAS+)
    """
    assert res in ALLEN_RESOLUTIONS, f"'res' must be in {ALLEN_RESOLUTIONS}"
    res = int(res)
    directory = Path(directory)
    directory.absolute()
    directory.mkdir(exist_ok=True, parents=True)

    # Preparing the filenames
    nrrd_file = directory / f"allen_template_{res}.nrrd"
    nifti_file = directory / f"allen_template_{res}.nii.gz"

    # Downloading the template
    rpa = ReferenceSpaceApi(base_uri=str(directory))
    rpa.download_template_volume(resolution=res, file_name=nrrd_file)

    # Loading the nrrd file
    vol, header = nrrd.read(nrrd_file)
    # Resizing the volume
    vol_resized = skit.resize(vol, (164, 212, 158) )

    # Preparing the affine to align the template in the RAS+
    r_mm = res / 1e3
    affine = np.array([[ 0,      0,     r_mm, 0  ],
                       [ -r_mm,  0,     0,    0  ],
                       [ 0,     -r_mm,  0,    0  ],
                       [ 0,      0,     0,    1  ]])

    # Creating a identity matrix
    I4 = np.eye(4)
    # Set the resolution to 70
    I4 = I4 * 0.07

    # Preparing the affine to align the template in the RAS+
    new_r1 =  I4[2,:]
    new_r2 = -I4[0,:]
    new_r3 = -I4[1,:]

    I4[0,:] = new_r1
    I4[1,:] = new_r2
    I4[2,:] = new_r3

    # Creating transformation matrix
    tr = np.eye(4)
    txyz = np.array([-5.675, -8.79448, -8.450335, 0])
    tr[:,3] = tr[:,3] + txyz

    # Applying the transformation
    I4 = I4@tr

    # Saving the volume (nifti)
    #img = nib.Nifti1Image(vol, affine)
    img = nib.Nifti1Image(vol_resized, I4)

    #nib.save(img, nifti_file)
    nib.save(img, nifti_file)

    # Deleting the nrrd file
    os.remove(nrrd_file)

    return nifti_file

download_allen_template(res=50, directory='/Users/mahdi/stage-2022-mahdi/src')

2022-05-03 14:59:54,068 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_50.nrrd


PosixPath('/Users/mahdi/stage-2022-mahdi/src/allen_template_50.nii.gz')